In [2]:
!pip -qq install langchain langchain-community faiss-cpu unstructured[pdf] sentence_transformers

In [4]:
import os
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain_community.llms import Ollama

In [5]:
llm = Ollama(model="llama3")

In [6]:
pdf_paths = [
    'uber-10-k-2023.pdf',
    'tsla-20231231-gen.pdf',
    'goog-10-k-2023 (1).pdf'
]

In [7]:
all_chunks = []

In [8]:
for pdf_path in pdf_paths:
    loader = UnstructuredPDFLoader(pdf_path)
    documents = loader.load()
    text_splitter = CharacterTextSplitter(
        separator='\n',
        chunk_size=3500,
        chunk_overlap=1000
    )
    text_chunks = text_splitter.split_documents(documents)
    all_chunks.extend(text_chunks)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [9]:
embeddings = HuggingFaceEmbeddings()
knowledge_base = FAISS.from_documents(all_chunks, embeddings)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=knowledge_base.as_retriever()
)

In [11]:
question = "What are the differences in the business of Tesla and Uber?"
response = qa_chain.invoke(question)

In [12]:
print(response['result'])

Based on the provided context, here are some key differences between the businesses of Tesla and Uber:

1. Primary Focus:
	* Tesla is primarily a electric vehicle manufacturer, with a focus on developing and selling electric cars.
	* Uber is a technology company that provides ride-sharing and delivery services, as well as freight and logistics solutions.

2. Autonomous Vehicle Development:
	* Tesla is known for its autonomous driving technology, which is integrated into their vehicles. They are also working on developing full self-driving (FSD) capabilities.
	* Uber has invested in companies like Aurora to develop autonomous vehicle technologies for its platform. However, it seems that they have not yet commercialized autonomous ride-hailing services.

3. Business Segments:
	* Tesla has a single operating segment: Automotive. They also generate revenue from energy generation and storage, as well as other sources.
	* Uber has three operating segments: Mobility, Delivery, and Freight. Ea